In [0]:
%load_ext autoreload
%autoreload 2
# Enables autoreload; learn more at https://docs.databricks.com/en/files/workspace-modules.html#autoreload-for-python-modules
# To disable autoreload; run %autoreload 0

In [0]:
# Ajouter le repo au Python Path
import sys
sys.path.append("/Workspace/Users/mandu543@gmail.com/databricks-movies-analytics/Movies_Project")

from src.utils import *
from pyspark.sql.functions import col, concat_ws
from delta.tables import DeltaTable

# Create schema if not exists
spark.sql(
    f"CREATE SCHEMA IF NOT EXISTS {SILVER_TABLE.split('.')[1]}"
)

In [0]:
print(SILVER_TABLE.split('.')[1])

## Intégration des données dans la couche silver

**Harmonisation des données depuis la couche bronze**

In [0]:
# Lire les données Bronze

df_bronze = spark.table(BRONZE_TABLE)


In [0]:
display(df_bronze)

In [0]:
# Nettoyage
df_bronze_clean = typage_data(df_bronze)

# Supprime les colonnes multiples et non utilisées
col_delete = ['backdrop_path', 'tagline', 'homepage', 'imdb_id', 'poster_path', 'genres', 'production_companies', 'production_countries', 'spoken_languages', 'keywords']
silver_movies = drop_columns(df_bronze_clean,col_delete)

display(silver_movies)

**Alimentation de la table de fait fact_movies**

In [0]:
# Vérifier si la colonne 'id' existe
if 'id' not in silver_movies.columns:
    # Créer un ID unique basé sur title + release_date
    silver_movies = silver_movies.withColumn("id", concat_ws("_", col("title"), col("release_date").cast("string")))
    print("⚠️ Colonne 'id' inexistante. Création de 'mon_id' unique automatiquement.")

# Merge incrémental dans Silver
try:
    delta_table = DeltaTable.forName(spark, SILVER_TABLE)

    # Compter lignes avant merge
    count_before = delta_table.toDF().count()

    # inserer les nouvelles lignes
    delta_table.alias("silver").merge(
        silver_movies.alias("bronze"),
        "silver.id = bronze.id"
    ).whenNotMatchedInsertAll().execute()

    print("✅ Merge incrémental terminé dans Silver")
    # Compter lignes après merge
    count_after = delta_table.toDF().count()
    print(f"Lignes insérées : {count_after - count_before}")
except:
    # Si la table n'existe pas encore, on la crée
    silver_movies.write.format("delta").mode("overwrite").saveAsTable(SILVER_TABLE)
    print("✅ Table Silver créée pour la première fois")

**Normalisation des tables relationnelles depuis le dataframe silver_films**

- Normalisation par genres

In [0]:
silver_movies_genre = create_silver_multivalue(df_bronze,"id", "genres")

In [0]:
display(silver_movies_genre)

In [0]:
# Insertion dans le catalog
try: 
    table_name = SILVER_ZONE + "." + "silver_movies_genre"
    save_dataframe(silver_movies_genre, table_name)

    print("✅ Table silver_movies_genre créée ")

except:
    print("Erreur : Alimentation de la table silver_movies_genre.")
